Best Model for Credit Card Approval Project

-- by **Gurnoordeep Singh Randhawa** 

This Project is based on a hypothetical situation which follows - <br>
The Bank provides us with a really small fraction of data from there database and wants us to -
* Clean the datasets and join them.
* Visualize on that data to make it presentablle and understandable to the board of directores.
* Find the fastest and the most accurate Model to find the features that are needed to be accessed and in which order while approoving a Credit for a new customer.

##Importing all the Libraries required.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

##Understanding Datasets

Firstly we will go ahead and access our data that we are using.
From pre assessment of the data we found that - 
* Both the files are in CSV format.
* The fields are seperated by the ',' deliminater.

###Application Records (df1)

This data set is the Application records. It contains all the information on all the clients and these clients are unique.<br>

In [ ]:
df1 = pd.read_csv('/kaggle/input/credit-card-approval-prediction/application_record.csv', delimiter=',', index_col=False)
df1.head()

In [ ]:
print('The shape of Application Record dataset is ' + str(df1.shape) +'.')
print('There are ' + str(df1['ID'].nunique()) + ' unique IDs in this dataset.')

In [ ]:
df1.isnull().sum()

To get the further info on our dataset we can see below.

In [ ]:
df1.head().T

###Credit Records (df2)

This dataset is called the Credit Records. <br>
It contains the status of each customers credit status and the count of months for.

In [ ]:
df2 = pd.read_csv('/kaggle/input/credit-card-approval-prediction/credit_record.csv', delimiter=',', index_col=False)
df2.head()

In [ ]:
print('The shape of Credit Record dataset is ' + str(df2.shape) +'.')
print('There are ' + str(df2['ID'].nunique()) + ' unique IDs in this dataset, which is much less than the parent dataset.')

##Merging and Cleaning the datasets

###Merging

Now we are going to merge the two datasets on the common column i.e. ID.

In [ ]:
df3 = pd.merge(df1, df2, on='ID')
df3.head()

In [ ]:
print('The shape of the merged dataset is ' + str(df3.shape) +'.')

###Cleaning the dataset

But this is not our final dataset as this dataset contains a lot of dupluicates for every time the customer missed a transaction. So -
* We are going to drop all the rows with the missing values.
* Drop all the duplicate rows and keep the first entry of the customer.
* We will do this on a copy of our merged dataset so that the original stays intact for further use.

In [ ]:
df4 = df3.dropna().drop_duplicates(subset='ID', keep='first')
df4.head()

Now this is the data set that we will do our visualization on.

In [ ]:
print('The shape of the cleaned data set is ' + str(df4.shape) +'.')

##Visualization on the cleaned dataset.

Now we are going to do a bit of visualization of the datt that we have so that our dataset is presentable to the board of directors.

###Age VS Customer Frequency

The age of the customers are given in a format which is little complicated. It is given as the amount of days from birth till today and the -ve sign indicated that the time is in past.<br>
So, we are going to convert this into the years and then map a histogram chart that tells us to]he amount of customer and which age bracket they are in.

In [ ]:
# Converting the Days of Birth field to Age in years.
df4['Age']=-(df4['DAYS_BIRTH'])//365	
print(df4['Age'].value_counts(bins=10,normalize=False,sort=False))
df4['Age'].plot(kind='hist',bins=20,density=False)

###Employment time VS Customer Frequency

Employment column is represented in the same format and we are going to covvert it the same way we did for the age.<br>
Then, we map a histogram representing the data of amount of customers vs the years they have been employed.

In [ ]:
# Converting the Days_Employed field from days to Years and the visualising.
df4['Employed']=-(df4['DAYS_EMPLOYED'])//365	
df4[df4['Employed']<0] = np.nan # replace by na
df4['DAYS_EMPLOYED']
df4['Employed'].fillna(df4['Employed'].mean(),inplace=False) #replace na by mean
df4['Employed'].plot(kind='hist',bins=20,density=False)

###Various Occupation Classifications

Taking a rough look on the Occupation field brfore visualising

In [ ]:
df4['OCCUPATION_TYPE'].value_counts()

In [ ]:
df4['OCCUPATION_TYPE'].value_counts().sort_values().plot(kind='barh', figsize=(9,12))

In [ ]:
PNoLoan = (df4.loc[df4['STATUS'] == 'X'].shape[0]/df4.shape[0]) * 100
print(str(round(PNoLoan, 2)) + "% of people have no loans.")

In [ ]:
PPaidOf = (df4.loc[df4['STATUS'] == 'C'].shape[0]/df4.shape[0]) * 100
print(str(round(PPaidOf, 2)) + "% of customers have paid off there dues.")

Now we will declare the people who have not paid there dues as the defaulters. These are represented by numbers in the Status field of the Credit Records dataset.

In [ ]:
defaulters= ['0','1','2','3','4','5']

In [ ]:
df4['OverDues'] = np.where(df4.STATUS.isin(defaulters), 1, 0)
df4.head()

In [ ]:
df4.groupby('OCCUPATION_TYPE')

###Defaulters VS PaidOff

This chart will represent the amount of money made by People who have paid of vs the Defaulters.

In [ ]:
plt.figure(figsize=(10,6))
df4[df4['OverDues']==1]['AMT_INCOME_TOTAL'].hist(alpha=0.7,color='Red', bins=6,label='Defaulters')
df4[df4['OverDues']==0]['AMT_INCOME_TOTAL'].hist(alpha=0.3,color='Blue', bins=6,label='PaidOff')
                                                    
plt.title('Customers around the income groups')                                                    
plt.legend()
plt.xlabel('Annual Income')
plt.ylabel('Number of Customers')

In [ ]:
df4['OverDues'].value_counts()

###Female VS Male

This chart tell us the Male and Female numbers in eeach category.

In [ ]:
pd.crosstab(df4.CODE_GENDER,df4.OverDues).plot(kind='bar')
plt.title('Gender vs Defaulters')
plt.xlabel('Gender')
plt.ylabel('Number of Customers')
plt.legend(labels = ["PaidOff", "Defaulters"])
plt.xticks(np.linspace(0, 1, 2), ['Female','Male'])

###Income types VS Customer Frequency

This chart tells us the income types of the people in both categories.

In [ ]:
pd.crosstab(df4.NAME_INCOME_TYPE, df4.OverDues).plot(kind='bar', figsize=(12,6))
plt.title('Customers across various Income Types')
plt.xlabel('Income Type')
plt.ylabel('Number of Customers')
plt.legend(labels = ["PaidOff", "Defaulters"])

## Correlations

Now we will find the feature correlation by mapping them.

###Heat Mapping

Now we are going to find the corelation between features for more in-depth review of how features work.<br>
The lesser they are correlated the better this is.

In [ ]:
# Put in the columns that we want to make the dummie variable of...
df7 = pd.get_dummies(df3, columns=['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE'], drop_first = False)
df7['OverDues'] = np.where(df7.STATUS.isin(defaulters), 1, 0)

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))

corr = df7[['OverDues', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'AMT_INCOME_TOTAL', 'FLAG_PHONE', 'CNT_FAM_MEMBERS']].corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns,
        annot=True, cmap=plt.cm.Reds)

###Pairplotting

Now we will do a bit of Pairplottin which is better to see the distribution of the single variable and its relation with other variable.<br>
This will help us identify trends for follow-up analysis.

In [ ]:
sns.pairplot(df7[['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'CNT_FAM_MEMBERS', 'OverDues']])

##Prediction Modelling

In [ ]:
#Making a copy of this df just in case we want to retry from the start
df5 = df4.copy()

Now we have columns with more than one catagorical values and there are many of them. <br>
If we go on converting each and every one of them into different columns with two categories it will take a lot of time. <br>
So, to simplify this we will use **get_dummies** cmd and mention the colmumns that we want to make dummie vaiables of.<br>
We do this to make as many features as possible for our prediction model as we cannot use string for analysis.

###Creating Dummie Variables

In [ ]:
df7.head()

###Declaring Features

In [ ]:
df7.columns

In [ ]:
#Mentioning our Features
Features = ['CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE',
       'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS',
       'CODE_GENDER_F', 'CODE_GENDER_M', 'FLAG_OWN_CAR_N', 'FLAG_OWN_CAR_Y',
       'FLAG_OWN_REALTY_N', 'FLAG_OWN_REALTY_Y',
       'NAME_INCOME_TYPE_Commercial associate', 'NAME_INCOME_TYPE_Pensioner',
       'NAME_INCOME_TYPE_State servant', 'NAME_INCOME_TYPE_Student',
       'NAME_INCOME_TYPE_Working', 'NAME_EDUCATION_TYPE_Academic degree',
       'NAME_EDUCATION_TYPE_Higher education',
       'NAME_EDUCATION_TYPE_Incomplete higher',
       'NAME_EDUCATION_TYPE_Lower secondary',
       'NAME_EDUCATION_TYPE_Secondary / secondary special',
       'NAME_FAMILY_STATUS_Civil marriage', 'NAME_FAMILY_STATUS_Married',
       'NAME_FAMILY_STATUS_Separated',
       'NAME_FAMILY_STATUS_Single / not married', 'NAME_FAMILY_STATUS_Widow',
       'NAME_HOUSING_TYPE_Co-op apartment',
       'NAME_HOUSING_TYPE_House / apartment',
       'NAME_HOUSING_TYPE_Municipal apartment',
       'NAME_HOUSING_TYPE_Office apartment',
       'NAME_HOUSING_TYPE_Rented apartment', 'NAME_HOUSING_TYPE_With parents']

###Train Test split

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split

train, test = train_test_split(df7, test_size = 0.3, random_state=21)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

features_train = train[Features]
label_train = train['OverDues']
features_test = test[Features]
label_test = test['OverDues']

In [ ]:
#df7.drop('OCCUPATION_TYPE', axis=1, inplace=True)

As our Employer wants us to find the most important feature for them to approove the credit for there customers, we have these two predictive models that we are going to put to test.

## Random Forest Classifier

This Model test all the features in multiple times in different orders. This is the reason that it is the slowest one but in theory oit should be the most accurate one as well.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

clf.fit(features_train,label_train)

pred_train = clf.predict(features_train)
pred_test = clf.predict(features_test)

from sklearn.metrics import accuracy_score
RFCaccuracy_train = (accuracy_score(pred_train,label_train)) * 100
RFCaccuracy_test = (accuracy_score(pred_test,label_test)) * 100

In [ ]:
print('Training Accuracy is ' + str(round(RFCaccuracy_train, 4)) + '%')
print('Test Accuracy is ' +  str(round(RFCaccuracy_test, 4)) + '%')

Here is the confusion mattrix for this model.

In [ ]:
pd.crosstab(label_test,pd.Series(pred_test),rownames=['ACTUAL'],colnames=['PRED'])

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

print('Accuracy Score')
print(accuracy_score(label_test, pred_test),'\n')

print('Precision Score')
print(precision_score(label_test, pred_test,average = None),'\n')

print('Confusion Matrix')
array = confusion_matrix(label_test, pred_test)
columns = ['Non Defaulter','Defaulter'] 
print(pd.DataFrame(array,columns = columns, index = columns),'\n')

print('Classification Report')
print(classification_report(label_test, pred_test),'\n')

## Decision Tree Classifier

This Model is faster but not that accurate as it only test the features one time.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=0)
tree.fit(features_train, label_train)

# predict train set
pred_train=tree.predict(features_train)
# predict test set
pred_test=tree.predict(features_test)

from sklearn.metrics import accuracy_score
DTCaccuracy_train = (accuracy_score(pred_train,label_train)) * 100
DTCaccuracy_test = (accuracy_score(pred_test,label_test)) * 100

In [ ]:
print('Training Accuracy is ' + str(round(DTCaccuracy_train, 4)) + '%')
print('Test Accuracy is ' +  str(round(DTCaccuracy_test, 4)) + '%')

In [ ]:
pd.crosstab(label_test,pd.Series(pred_test),rownames=['ACTUAL'],colnames=['PRED'])

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

print('Accuracy Score')
print(accuracy_score(label_test, pred_test),'\n')

print('Precision Score')
print(precision_score(label_test, pred_test,average = None),'\n')

print('Confusion Matrix')
array = confusion_matrix(label_test, pred_test)
columns = ['Non Defaulter','Defaulter']  
print(pd.DataFrame(array,columns = columns, index = columns),'\n')

print('Classification Report')
print(classification_report(label_test, pred_test),'\n')

## Feature Importance

In [ ]:
dfz=pd.DataFrame({'features':features_train.columns,'importances':tree.feature_importances_})
dfz.sort_values('importances',inplace=True)
dfz.plot(kind='barh', title = 'Decision Tree Classifier\nFeature Importances', y='importances',x='features',color='brown', figsize=(12,22))

## Conclusion....

* From the model testing done above we can see that the Decision Tree Classifier is better faster and has almost the same accuracy as compared to the Random Forest Classifier. 
* This tells us that on per our employers demand the best fit for there work would be the Decision tree classifier and We did the feature importance chart on them.<br> 
* The most important features that are at the top of the chart gives the bank a slight insight on what to test for while approving a credit based on there previous customer data.